In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

%config Completer.use_jedi = False

In [2]:
from fugue_notebook import setup

C:\Users\phamp\AppData\Roaming\Python\Python39\site-packages\fugue_sql\__init__.py:8: UserWarning: fsql and FugueSQLWorkflow now should be imported directly from fugue, fugue_sql will be removed in 0.9.0
  warnings.warn(


In [3]:
setup()

<IPython.core.display.Javascript object>

In [4]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
%%fsql
SELECT * 
FROM iris
PRINT

,sepal_length:double,sepal_width:double,petal_length:double,petal_width:double,species:str
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


## Luu dữ liệu vào csv

In [6]:
%%fsql
SELECT *
FROM iris
SAVE OVERWRITE 'C:/data/iris.csv' (header = true)

## Lưu dữ liệu vào file json

In [7]:
%%fsql
SELECT *
FROM iris
SAVE OVERWRITE 'C:/data/iris.json' 

In [8]:
## Lưu dữ liệu vào file parquet

In [9]:
%%fsql
SELECT *
FROM iris
SAVE OVERWRITE 'C:/data/iris.parquet' 

In [10]:
%%fsql
SELECT *
FROM iris
SAVE OVERWRITE PARQUET 'C:/data/iris' 

## Lưu dữ liệu vào file json

In [11]:
%%fsql
SELECT *
FROM iris
SAVE OVERWRITE 'C:/data/iris.avro' 

_2 _State.RUNNING -> _State.FAILED  No module named 'pandavro'


ModuleNotFoundError: No module named 'pandavro'

In [ ]:
## Dọc dữ liệu từ file csv

In [ ]:
%%fsql
LOAD 'C:/data/iris.csv' (header = true, infer_schema = true)
PRINT

In [ ]:
%%fsql
LOAD 'C:/data/iris.csv' (header = true)
COLUMNS sepal_length:float,sepal_width:float,petal_length:float, petal_width:float, species:str
PRINT

In [ ]:
%%fsql
LOAD 'C:/data/iris.csv' (header = true)
COLUMNS sepal_length:float,sepal_width:float,petal_length:float, petal_width:float, species:str
YIELD DATAFRAME AS df

In [ ]:
df1 =df.as_pandas()

In [ ]:
df1

In [ ]:
df

## Đọc từ file json

In [ ]:
%%fsql
LOAD 'C:/data/iris.json' 
PRINT

## Đọc từ file Parquet

In [ ]:
%%fsql
LOAD 'C:/data/iris.parquet' 
PRINT

In [ ]:
%%fsql
LOAD PARQUET 'C:/data/iris' 
PRINT

In [ ]:
## đọc từ file avro

In [ ]:
%%fsql
LOAD 'C:/data/iris.avro' 
PRINT

## goi hàm Fugue SQL

In [ ]:
from typing import List, Any

In [ ]:
def tao_bang(lst1:List[int], lst2:List[Any]) -> List[List[Any]]:
    lst_kq =[]
    for i in range(len(lst1)):
        lst_kq.append([lst1[i],lst2[i]])
    return lst_kq

In [ ]:
%%fsql
CREATE USING tao_bang(lst1=[1,2], lst2 =['Hello','World'])
SCHEMA num:int, name:str
PRINT

In [ ]:
## định nghĩa cấu trúc cột lúc tạo bảng

In [ ]:
# schema: stt:int, ten:str
def tao_bang2(lst1:List[int], lst2:List[Any]) -> List[List[Any]]:
    lst_kq =[]
    for i in range(len(lst1)):
        lst_kq.append([lst1[i],lst2[i]])
    return lst_kq

In [ ]:
%%fsql
CREATE USING tao_bang2(lst1=[1,2], lst2 =['Hello','World'])
PRINT

In [ ]:
## tạo hàm trả về dataframe

In [ ]:
def tao_bang3(lst1:List[int], lst2:List[Any]) -> pd.DataFrame:
    df = pd.DataFrame({
        'stt':lst1,
        'ten':lst2
    })
    return df

In [ ]:
%%fsql
CREATE USING tao_bang3(lst1=[1,2], lst2 =['Hello','World'])
PRINT

## Trực quan hóa dữ liệu với Fugue SQL

In [ ]:
%%fsql
LOAD 'C:/data/covid19.parquet' 
PRINT

In [ ]:
# xây dựng hàm vẽ biểu đồ lineplot
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def vebieudo(df:pd.DataFrame, x_col:str, y_col:str, title:str, x_title:str, y_title:str) -> None:
    plt.figure(figsize=(10,6))
    sns.lineplot(data=df,x=x_col, y=y_col)
    plt.xlabel(x_title)
    plt.ylabel(y_title)
    plt.title(title)
    plt.show()

In [ ]:
%%fsql
df = LOAD 'C:/data/covid_19_data.parquet' 
SELECT * 
WHERE Country = 'Vietnam'
PRINT

In [ ]:
%%fsql
df = LOAD 'C:/data/covid_19_data.parquet' 
SELECT * 
WHERE Country = 'Vietnam'
OUTPUT USING vebieudo(x_col='ObservationDate', y_col = 'Confirmed',
                     title= 'Tổng số ca bệnh ở Việt Nam', x_title = 'Ngày', y_title = 'Tổng số ca khỏi bệnh')

In [ ]:
%%fsql
df = LOAD 'C:/data/covid_19_data.parquet' 
SELECT * 
WHERE Country = 'Vietnam'
OUTPUT USING vebieudo(x_col='ObservationDate', y_col = 'Recovered',
                     title= 'Tổng số ca khỏi bệnh ở Việt Nam', x_title = 'Ngày', y_title = 'Tổng số ca khỏi bệnh')

## Transform

In [21]:
df = pd.read_csv('data/insurance.csv')

In [22]:
from typing import Iterable, Dict, Any

In [23]:
# schema: *, gender:str
def taocotgender(data:Iterable[Dict[str,Any]]) -> Iterable[Dict[str,Any]]:
    for row in data:
        row['gender'] = 1 if row['sex'] == 'female' else 0
        yield row

In [24]:
%%fsql
SELECT *
FROM df
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


In [25]:
%%fsql
SELECT *
FROM df
TRANSFORM USING taocotgender
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double,gender:str
0,19,female,27.900,0,yes,southwest,16884.92400,1
1,18,male,33.770,1,no,southeast,1725.55230,0
2,28,male,33.000,3,no,southeast,4449.46200,0
3,33,male,22.705,0,no,northwest,21984.47061,0
4,32,male,28.880,0,no,northwest,3866.85520,0
5,31,female,25.740,0,no,southeast,3756.62160,1
6,46,female,33.440,1,no,southeast,8240.58960,1
7,37,female,27.740,3,no,northwest,7281.50560,1
8,37,male,29.830,2,no,northeast,6406.41070,0
9,60,female,25.840,0,no,northwest,28923.13692,1


In [28]:
# schema: *, age_gender:str
def taocotagegender(data:pd.DataFrame, phancach:str) -> pd.DataFrame:
    data['age_gender'] = data['age'].astype(str) + phancach + data['sex']
    return data    

In [29]:
%%fsql
SELECT *
FROM df
TRANSFORM USING taocotagegender(phancach='_')
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double,age_gender:str
0,19,female,27.900,0,yes,southwest,16884.92400,19_female
1,18,male,33.770,1,no,southeast,1725.55230,18_male
2,28,male,33.000,3,no,southeast,4449.46200,28_male
3,33,male,22.705,0,no,northwest,21984.47061,33_male
4,32,male,28.880,0,no,northwest,3866.85520,32_male
5,31,female,25.740,0,no,southeast,3756.62160,31_female
6,46,female,33.440,1,no,southeast,8240.58960,46_female
7,37,female,27.740,3,no,northwest,7281.50560,37_female
8,37,male,29.830,2,no,northeast,6406.41070,37_male
9,60,female,25.840,0,no,northwest,28923.13692,60_female


## Processor

In [34]:
def gopdata(data1:pd.DataFrame, data2:pd.DataFrame) -> pd.DataFrame:
    dfTemp = pd.concat([data1,data2], ignore_index=True)
    return dfTemp

In [35]:
%%fsql
df1 = SELECT * FROM df WHERE children = 0 LIMIT 3
df2 = SELECT * FROM df WHERE children = 1 LIMIT 3
PROCESS df1, df2 USING gopdata
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
0,19,female,27.900,0,yes,southwest,16884.92400
1,33,male,22.705,0,no,northwest,21984.47061
2,32,male,28.880,0,no,northwest,3866.85520
3,18,male,33.770,1,no,southeast,1725.55230
4,46,female,33.440,1,no,southeast,8240.58960
5,19,male,24.600,1,no,southwest,1837.23700
